In [ ]:
from google.colab import drive
drive.mount('/content/drive')

project_dir = '/content/drive/My Drive/Colab Notebooks/event_camera_repo'
%cd {project_dir}

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/event_camera_repo'
/content


In [2]:
!pip install hydra-core omegaconf hdf5plugin h5py numba imageio imageio-ffmpeg tqdm torchvision --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.5 MB/s eta 0:00:00


In [3]:
import torch
import hydra
from omegaconf import DictConfig
from torch.utils.data import DataLoader
import random
import numpy as np
from enum import Enum, auto
from tqdm import tqdm
from pathlib import Path
from typing import Dict, Any
import os
import time

import math
from pathlib import PurePath
from typing import Tuple
import cv2
import hdf5plugin
import h5py
from numba import jit
import imageio
imageio.plugins.freeimage.download()
import imageio.v3 as iio
from torchvision.transforms import RandomCrop
from torchvision import transforms as tf
from torch.utils.checkpoint import checkpoint
from torch.utils.data import Dataset
import torchvision.transforms.functional as F

from torch import nn

Imageio: 'libfreeimage-3.16.0-linux64.so' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/freeimage/libfreeimage-3.16.0-linux64.so (4.6 MB)
Downloading: 8192/4830080 bytes (0.2%)90112/4830080 bytes (1.9%)647168/4830080 bytes (13.4%)4603904/4830080 bytes (95.3%)4830080/4830080 bytes (100.0%)
  Done
File saved as /root/.imageio/freeimage/libfreeimage-3.16.0-linux64.so.


In [ ]:
!pip install einops
from einops.layers.torch import Rearrange
from einops import rearrange

In [ ]:
from omegaconf import OmegaConf
from hydra import compose, initialize

cfg_string = '''
dataset_path: data
seed: 42
num_epoch: 100
data_loader:
    common:
        num_voxel_bins: 15
    train:
        batch_size: 16
        shuffle: false
    test:
        batch_size: 1
        shuffle: false
train:
    no_batch_norm: false
    initial_learning_rate: 0.01
    learning_rate_decay: 0.9
    weight_decay: 0.0001
    epochs: 10
'''
cfg = OmegaConf.create(cfg_string)

## `utils.py`

In [ ]:
def set_seed(seed: int = 0) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

class RepresentationType(Enum):
    VOXEL = auto()
    STEPAN = auto()


class EventRepresentation:
    def __init__(self):
        pass

    def convert(self, events):
        raise NotImplementedError


class VoxelGrid(EventRepresentation):
    def __init__(self, input_size: tuple, normalize: bool):
        assert len(input_size) == 3
        self.voxel_grid = torch.zeros(
            (input_size), dtype=torch.float, requires_grad=False)
        self.nb_channels = input_size[0]
        self.normalize = normalize

    def convert(self, events):
        C, H, W = self.voxel_grid.shape
        with torch.no_grad():
            self.voxel_grid = self.voxel_grid.to(events['p'].device)
            voxel_grid = self.voxel_grid.clone()

            t_norm = events['t']
            t_norm = (C - 1) * (t_norm-t_norm[0]) / (t_norm[-1]-t_norm[0])

            x0 = events['x'].int()
            y0 = events['y'].int()
            t0 = t_norm.int()

            value = 2*events['p']-1
            #start_t = time()
            for xlim in [x0, x0+1]:
                for ylim in [y0, y0+1]:
                    for tlim in [t0, t0+1]:

                        mask = (xlim < W) & (xlim >= 0) & (ylim < H) & (
                            ylim >= 0) & (tlim >= 0) & (tlim < self.nb_channels)
                        interp_weights = value * (1 - (xlim-events['x']).abs()) * (
                            1 - (ylim-events['y']).abs()) * (1 - (tlim - t_norm).abs())
                        index = H * W * tlim.long() + \
                            W * ylim.long() + \
                            xlim.long()

                        voxel_grid.put_(
                            index[mask], interp_weights[mask], accumulate=True)

            if self.normalize:
                mask = torch.nonzero(voxel_grid, as_tuple=True)
                if mask[0].size()[0] > 0:
                    mean = voxel_grid[mask].mean()
                    std = voxel_grid[mask].std()
                    if std > 0:
                        voxel_grid[mask] = (voxel_grid[mask] - mean) / std
                    else:
                        voxel_grid[mask] = voxel_grid[mask] - mean

        return voxel_grid


class PolarityCount(EventRepresentation):
    def __init__(self, input_size: tuple):
        assert len(input_size) == 3
        self.voxel_grid = torch.zeros(
            (input_size), dtype=torch.float, requires_grad=False)
        self.nb_channels = input_size[0]

    def convert(self, events):
        C, H, W = self.voxel_grid.shape
        with torch.no_grad():
            self.voxel_grid = self.voxel_grid.to(events['p'].device)
            voxel_grid = self.voxel_grid.clone()

            x0 = events['x'].int()
            y0 = events['y'].int()

            #start_t = time()
            for xlim in [x0, x0+1]:
                for ylim in [y0, y0+1]:
                    mask = (xlim < W) & (xlim >= 0) & (ylim < H) & (
                        ylim >= 0)
                    interp_weights = (1 - (xlim-events['x']).abs()) * (
                        1 - (ylim-events['y']).abs())
                    index = H * W * events['p'].long() + \
                        W * ylim.long() + \
                        xlim.long()

                    voxel_grid.put_(
                        index[mask], interp_weights[mask], accumulate=True)

        return voxel_grid


def flow_16bit_to_float(flow_16bit: np.ndarray):
    assert flow_16bit.dtype == np.uint16
    assert flow_16bit.ndim == 3
    h, w, c = flow_16bit.shape
    assert c == 3

    valid2D = flow_16bit[..., 2] == 1
    assert valid2D.shape == (h, w)
    assert np.all(flow_16bit[~valid2D, -1] == 0)
    valid_map = np.where(valid2D)

    # to actually compute something useful:
    flow_16bit = flow_16bit.astype('float')

    flow_map = np.zeros((h, w, 2))
    flow_map[valid_map[0], valid_map[1], 0] = (
        flow_16bit[valid_map[0], valid_map[1], 0] - 2 ** 15) / 128
    flow_map[valid_map[0], valid_map[1], 1] = (
        flow_16bit[valid_map[0], valid_map[1], 1] - 2 ** 15) / 128
    return flow_map, valid2D

In [ ]:
def warp_images_with_flow(images, flow):
    dim3 = 0
    if images.dim() == 3:
        dim3 = 1
        images = images.unsqueeze(0)
        flow = flow.unsqueeze(0)
    height = images.shape[2]
    width = images.shape[3]
    flow_x,flow_y = flow[:,0,...],flow[:,1,...]
    coord_x, coord_y = torch.meshgrid(torch.arange(height), torch.arange(width))

    if torch.cuda.is_available():
        pos_x = coord_x.reshape(height,width).type(torch.float32).cuda() + flow_x
        pos_y = coord_y.reshape(height,width).type(torch.float32).cuda() + flow_y
    else: # Troubleshoot without cuda
        pos_x = coord_x.reshape(height,width).type(torch.float32) + flow_x
        pos_y = coord_y.reshape(height,width).type(torch.float32) + flow_y
    pos_x = (pos_x-(height-1)/2)/((height-1)/2)
    pos_y = (pos_y-(width-1)/2)/((width-1)/2)

    pos = torch.stack((pos_y,pos_x),3).type(torch.float32)
    result = torch.nn.functional.grid_sample(images, pos, mode='bilinear', padding_mode='zeros')
    if dim3 == 1:
        result = result.squeeze()

    return result

def charbonnier_loss(delta, alpha=0.45, epsilon=1e-3):
        loss = torch.mean(torch.pow((delta ** 2 + epsilon ** 2), alpha))
        return loss

def compute_smoothness_loss(flow):

    flow_ucrop = flow[..., 1:]
    flow_dcrop = flow[..., :-1]
    flow_lcrop = flow[..., 1:, :]
    flow_rcrop = flow[..., :-1, :]

    flow_ulcrop = flow[..., 1:, 1:]
    flow_drcrop = flow[..., :-1, :-1]
    flow_dlcrop = flow[..., :-1, 1:]
    flow_urcrop = flow[..., 1:, :-1]

    smoothness_loss = charbonnier_loss(flow_lcrop - flow_rcrop) +\
                      charbonnier_loss(flow_ucrop - flow_dcrop) +\
                      charbonnier_loss(flow_ulcrop - flow_drcrop) +\
                      charbonnier_loss(flow_dlcrop - flow_urcrop)
    smoothness_loss /= 4.

    return smoothness_loss

def compute_photometric_loss(prev_images, next_images, flow_dict):
    total_photometric_loss = 0.
    loss_weight_sum = 0.
    for i in range(len(flow_dict)):
        for image_num in range(prev_images.shape[0]):
            flow = flow_dict["flow{}".format(i)][image_num]
            height = flow.shape[1]
            width = flow.shape[2]

            prev_images_resize = F.to_tensor(F.resize(F.to_pil_image(prev_images[image_num].cpu()),
                                                    [height, width]))
            next_images_resize = F.to_tensor(F.resize(F.to_pil_image(next_images[image_num].cpu()),
                                                    [height, width]))

            if torch.cuda.is_available():
                prev_images_resize = prev_images_resize.cuda()
                next_images_resize = next_images_resize.cuda()

            next_images_warped = warp_images_with_flow(next_images_resize, flow)

            distance = next_images_warped - prev_images_resize
            photometric_loss = charbonnier_loss(distance)
            total_photometric_loss += photometric_loss
        loss_weight_sum += 1.
    total_photometric_loss /= loss_weight_sum

    return total_photometric_loss


class TotalLoss(torch.nn.Module):
    def __init__(self, smoothness_weight, weight_decay_weight=1e-4):
        super(TotalLoss, self).__init__()
        self._smoothness_weight = smoothness_weight
        self._weight_decay_weight = weight_decay_weight

    def forward(self, flow_dict, prev_image, next_image, EVFlowNet_model):
        # weight decay loss
        weight_decay_loss = 0
        for i in EVFlowNet_model.parameters():
            weight_decay_loss += torch.sum(i ** 2) / 2 * self._weight_decay_weight

        # smoothness loss
        smoothness_loss = 0
        for i in range(len(flow_dict)):
            smoothness_loss += compute_smoothness_loss(flow_dict["flow{}".format(i)])
        smoothness_loss *= self._smoothness_weight / 4.

        # Photometric loss.
        photometric_loss = compute_photometric_loss(prev_image,
                                                    next_image,
                                                    flow_dict)

        # Warped next image for debugging.
        #next_image_warped = warp_images_with_flow(next_image,
        #                                          flow_dict['flow3'])

        loss = weight_decay_loss + photometric_loss + smoothness_loss

        return loss

## `datasets.py`

In [ ]:
VISU_INDEX = 1


class EventSlicer:
    def __init__(self, h5f: h5py.File):
        self.h5f = h5f

        self.events = dict()
        for dset_str in ['p', 'x', 'y', 't']:
            self.events[dset_str] = self.h5f['events/{}'.format(dset_str)]

        # This is the mapping from milliseconds to event index:
        # It is defined such that
        # (1) t[ms_to_idx[ms]] >= ms*1000
        # (2) t[ms_to_idx[ms] - 1] < ms*1000
        # ,where 'ms' is the time in milliseconds and 't' the event timestamps in microseconds.
        #
        # As an example, given 't' and 'ms':
        # t:    0     500    2100    5000    5000    7100    7200    7200    8100    9000
        # ms:   0       1       2       3       4       5       6       7       8       9
        #
        # we get
        #
        # ms_to_idx:
        #       0       2       2       3       3       3       5       5       8       9
        self.ms_to_idx = np.asarray(self.h5f['ms_to_idx'], dtype='int64')

        self.t_offset = int(h5f['t_offset'][()])
        self.t_final = int(self.events['t'][-1]) + self.t_offset

    def get_final_time_us(self):
        return self.t_final

    def get_events(self, t_start_us: int, t_end_us: int) -> Dict[str, np.ndarray]:
        """Get events (p, x, y, t) within the specified time window
        Parameters
        ----------
        t_start_us: start time in microseconds
        t_end_us: end time in microseconds
        Returns
        -------
        events: dictionary of (p, x, y, t) or None if the time window cannot be retrieved
        """
        assert t_start_us < t_end_us

        # We assume that the times are top-off-day, hence subtract offset:
        t_start_us -= self.t_offset
        t_end_us -= self.t_offset

        t_start_ms, t_end_ms = self.get_conservative_window_ms(
            t_start_us, t_end_us)
        t_start_ms_idx = self.ms2idx(t_start_ms)
        t_end_ms_idx = self.ms2idx(t_end_ms)
        if t_start_ms_idx is None or t_end_ms_idx is None:
            print('Error', 'start', t_start_us, 'end', t_end_us)
            # Cannot guarantee window size anymore
            return None

        events = dict()
        time_array_conservative = np.asarray(
            self.events['t'][t_start_ms_idx:t_end_ms_idx])
        idx_start_offset, idx_end_offset = self.get_time_indices_offsets(
            time_array_conservative, t_start_us, t_end_us)
        t_start_us_idx = t_start_ms_idx + idx_start_offset
        t_end_us_idx = t_start_ms_idx + idx_end_offset
        # Again add t_offset to get gps time
        events['t'] = time_array_conservative[idx_start_offset:idx_end_offset] + self.t_offset
        for dset_str in ['p', 'x', 'y']:
            events[dset_str] = np.asarray(
                self.events[dset_str][t_start_us_idx:t_end_us_idx])
            assert events[dset_str].size == events['t'].size
        return events

    @staticmethod
    def get_conservative_window_ms(ts_start_us: int, ts_end_us) -> Tuple[int, int]:
        """Compute a conservative time window of time with millisecond resolution.
        We have a time to index mapping for each millisecond. Hence, we need
        to compute the lower and upper millisecond to retrieve events.
        Parameters
        ----------
        ts_start_us:    start time in microseconds
        ts_end_us:      end time in microseconds
        Returns
        -------
        window_start_ms:    conservative start time in milliseconds
        window_end_ms:      conservative end time in milliseconds
        """
        assert ts_end_us > ts_start_us
        window_start_ms = math.floor(ts_start_us/1000)
        window_end_ms = math.ceil(ts_end_us/1000)
        return window_start_ms, window_end_ms

    @staticmethod
    @jit(nopython=True)
    def get_time_indices_offsets(
            time_array: np.ndarray,
            time_start_us: int,
            time_end_us: int) -> Tuple[int, int]:
        """Compute index offset of start and end timestamps in microseconds
        Parameters
        ----------
        time_array:     timestamps (in us) of the events
        time_start_us:  start timestamp (in us)
        time_end_us:    end timestamp (in us)
        Returns
        -------
        idx_start:  Index within this array corresponding to time_start_us
        idx_end:    Index within this array corresponding to time_end_us
        such that (in non-edge cases)
        time_array[idx_start] >= time_start_us
        time_array[idx_end] >= time_end_us
        time_array[idx_start - 1] < time_start_us
        time_array[idx_end - 1] < time_end_us
        this means that
        time_start_us <= time_array[idx_start:idx_end] < time_end_us
        """

        assert time_array.ndim == 1

        idx_start = -1
        if time_array[-1] < time_start_us:

            # Return same index twice: array[x:x] is empty.
            return time_array.size, time_array.size
        else:
            for idx_from_start in range(0, time_array.size, 1):
                if time_array[idx_from_start] >= time_start_us:
                    idx_start = idx_from_start
                    break
        assert idx_start >= 0

        idx_end = time_array.size
        for idx_from_end in range(time_array.size - 1, -1, -1):
            if time_array[idx_from_end] >= time_end_us:
                idx_end = idx_from_end
            else:
                break

        assert time_array[idx_start] >= time_start_us
        if idx_end < time_array.size:
            assert time_array[idx_end] >= time_end_us
        if idx_start > 0:
            assert time_array[idx_start - 1] < time_start_us
        if idx_end > 0:
            assert time_array[idx_end - 1] < time_end_us
        return idx_start, idx_end

    def ms2idx(self, time_ms: int) -> int:
        assert time_ms >= 0
        if time_ms >= self.ms_to_idx.size:
            return None
        return self.ms_to_idx[time_ms]


class Sequence(Dataset):
    def __init__(self, seq_path: Path, representation_type: RepresentationType, mode: str = 'test', delta_t_ms: int = 100,
                 num_bins: int = 4, transforms=[], name_idx=0, visualize=False, load_gt=False):
        assert num_bins >= 1
        assert delta_t_ms == 100
        assert seq_path.is_dir(), seq_path
        assert mode in {'train', 'test'}
        assert representation_type is not None
        '''
        ディレクトリ構造:

        data
        ├─test
        |  ├─seq_1
        |  |    ├─events_left
        |  |    |   ├─events.h5
        |  |    |   └─rectify_map.h5
        |  |    └─forward_timestamps.txt
        └─train
            ├─seq_1
            |    ├─events_left
            |    |       ├─ events.h5
            |    |       └─ rectify_map.h5
            |    ├─flow_forward
            |    |       ├─ 000134.png
            |    |       |.....
            |    └─forward_timestamps.txt
            ├─seq_2
            └─seq_3
        '''
        self.seq_name = PurePath(seq_path).name
        self.mode = mode
        self.name_idx = name_idx
        self.visualize_samples = visualize
        self.load_gt = load_gt
        self.transforms = transforms
        if self.mode == "test":
            assert load_gt == False
            # Get Test Timestamp File
            ev_dir_location = seq_path / 'events_left'
            timestamp_file = seq_path / 'forward_timestamps.txt'
            flow_path = seq_path / 'flow_forward'
            timestamps_flow = np.loadtxt(
                seq_path / 'forward_timestamps.txt', delimiter=',', dtype='int64')
            self.indices = np.arange(len(timestamps_flow))
            self.timestamps_flow = timestamps_flow[:, 0]

        elif self.mode == "train":
            ev_dir_location = seq_path / 'events_left'
            flow_path = seq_path / 'flow_forward'
            timestamp_file = seq_path / 'forward_timestamps.txt'
            self.flow_png = [Path(os.path.join(flow_path, img)) for img in sorted(
                os.listdir(flow_path))]
            timestamps_flow = np.loadtxt(
                seq_path / 'forward_timestamps.txt', delimiter=',', dtype='int64')
            self.indices = np.arange(len(timestamps_flow))
            self.timestamps_flow = timestamps_flow[:, 0]
        else:
            pass
        assert timestamp_file.is_file()

        file = np.genfromtxt(
            timestamp_file,
            delimiter=','
        )

        self.idx_to_visualize = file[:, 2] if file.shape[1] == 3 else []

        # Save output dimensions
        self.height = 480
        self.width = 640
        self.num_bins = num_bins


        # Set event representation
        self.voxel_grid = VoxelGrid(
                (self.num_bins, self.height, self.width), normalize=True)
        self.delta_t_us = delta_t_ms * 1000

        # Left events only
        ev_data_file = ev_dir_location / 'events.h5'
        ev_rect_file = ev_dir_location / 'rectify_map.h5'

        h5f_location = h5py.File(str(ev_data_file), 'r')
        self.h5f = h5f_location
        self.event_slicer = EventSlicer(h5f_location)

        self.h5rect = h5py.File(str(ev_rect_file), 'r')
        self.rectify_ev_map = self.h5rect['rectify_map'][()]


    def events_to_voxel_grid(self, p, t, x, y, device: str = 'cpu'):
        t = (t - t[0]).astype('float32')
        t = (t/t[-1])
        x = x.astype('float32')
        y = y.astype('float32')
        pol = p.astype('float32')
        event_data_torch = {
            'p': torch.from_numpy(pol),
            't': torch.from_numpy(t),
            'x': torch.from_numpy(x),
            'y': torch.from_numpy(y),
        }
        return self.voxel_grid.convert(event_data_torch)

    def getHeightAndWidth(self):
        return self.height, self.width

    @staticmethod
    def get_disparity_map(filepath: Path):
        assert filepath.is_file()
        disp_16bit = cv2.imread(str(filepath), cv2.IMREAD_ANYDEPTH)
        return disp_16bit.astype('float32')/256

    @staticmethod
    def load_flow(flowfile: Path):
        assert flowfile.exists()
        assert flowfile.suffix == '.png'
        flow_16bit = iio.imread(str(flowfile), plugin='PNG-FI')
        flow, valid2D = flow_16bit_to_float(flow_16bit)
        return flow, valid2D

    @staticmethod
    def close_callback(h5f):
        h5f.close()

    def get_image_width_height(self):
        return self.height, self.width

    def __len__(self):
        # Ignore the first and last images as their own
        return len(self.timestamps_flow) # - 2

    def rectify_events(self, x: np.ndarray, y: np.ndarray):
        # assert location in self.locations
        # From distorted to undistorted
        rectify_map = self.rectify_ev_map
        assert rectify_map.shape == (
            self.height, self.width, 2), rectify_map.shape
        assert x.max() < self.width
        assert y.max() < self.height
        return rectify_map[y, x]

    def get_data(self, index) -> Dict[str, any]:
        # Adjust index to skip the first element
#         index += 1

        ts_start: int = self.timestamps_flow[index] - self.delta_t_us
        ts_end: int = self.timestamps_flow[index]

        file_index = self.indices[index]

        output = {
            'file_index': file_index,
            'timestamp': self.timestamps_flow[index],
            'seq_name': self.seq_name
        }
        # Save sample for benchmark submission
        output['save_submission'] = file_index in self.idx_to_visualize
        output['visualize'] = self.visualize_samples
        event_data = self.event_slicer.get_events(
            ts_start, ts_end)
        p = event_data['p']
        t = event_data['t']
        x = event_data['x']
        y = event_data['y']

        xy_rect = self.rectify_events(x, y)
        x_rect = xy_rect[:, 0]
        y_rect = xy_rect[:, 1]

        if self.voxel_grid is None:
            raise NotImplementedError
        else:
            event_representation = self.events_to_voxel_grid(
                p, t, x_rect, y_rect)
            output['event_volume'] = event_representation
        output['name_map'] = self.name_idx

        if self.load_gt:
            output['flow_gt'] = [torch.tensor(x) for x in self.load_flow(self.flow_png[index])]
            output['flow_gt'][0] = torch.moveaxis(output['flow_gt'][0], -1, 0)
            output['flow_gt'][1] = torch.unsqueeze(output['flow_gt'][1], 0)

            flow_gt_shape = [tensor.shape for tensor in output['flow_gt']]
            zero_flow_gt = [torch.zeros_like(tensor) for tensor in output['flow_gt']]

            # Load previous image
            if index > 0:
                output['prev_flow_gt'] = [torch.tensor(x) for x in self.load_flow(self.flow_png[index - 1])]
                output['prev_flow_gt'][0] = torch.moveaxis(output['prev_flow_gt'][0], -1, 0)
                output['prev_flow_gt'][1] = torch.unsqueeze(output['prev_flow_gt'][1], 0)
            else:
                output['prev_flow_gt'] = zero_flow_gt

            # Load next image
            if index < len(self.timestamps_flow) - 1:
                output['next_flow_gt'] = [torch.tensor(x) for x in self.load_flow(self.flow_png[index + 1])]
                output['next_flow_gt'][0] = torch.moveaxis(output['next_flow_gt'][0], -1, 0)
                output['next_flow_gt'][1] = torch.unsqueeze(output['next_flow_gt'][1], 0)
            else:
                output['next_flow_gt'] = zero_flow_gt

        return output

    def __getitem__(self, idx):
        # Adjust index to skip the first element
        sample = self.get_data(idx) # idx + 1

        if self.transforms:
            sample = self.transforms(sample)

        return sample

    def get_voxel_grid(self, idx):

        if idx == 0:
            event_data = self.event_slicer.get_events(
                self.timestamps_flow[0] - self.delta_t_us, self.timestamps_flow[0])
        elif idx > 0 and idx <= self.__len__():
            event_data = self.event_slicer.get_events(
                self.timestamps_flow[idx-1], self.timestamps_flow[idx-1] + self.delta_t_us)
        else:
            raise IndexError

        p = event_data['p']
        t = event_data['t']
        x = event_data['x']
        y = event_data['y']

        xy_rect = self.rectify_events(x, y)
        x_rect = xy_rect[:, 0]
        y_rect = xy_rect[:, 1]
        return self.events_to_voxel_grid(p, t, x_rect, y_rect)

    def get_event_count_image(self, ts_start, ts_end, num_bins, normalize=True):
        assert ts_end > ts_start
        delta_t_bin = (ts_end - ts_start) / num_bins
        ts_start_bin = np.linspace(
            ts_start, ts_end, num=num_bins, endpoint=False)
        ts_end_bin = ts_start_bin + delta_t_bin
        assert abs(ts_end_bin[-1] - ts_end) < 10.
        ts_end_bin[-1] = ts_end

        event_count = torch.zeros(
            (num_bins, self.height, self.width), dtype=torch.float, requires_grad=False)

        for i in range(num_bins):
            event_data = self.event_slicer.get_events(
                ts_start_bin[i], ts_end_bin[i])
            p = event_data['p']
            t = event_data['t']
            x = event_data['x']
            y = event_data['y']

            t = (t - t[0]).astype('float32')
            t = (t/t[-1])
            x = x.astype('float32')
            y = y.astype('float32')
            pol = p.astype('float32')
            event_data_torch = {
                'p': torch.from_numpy(pol),
                't': torch.from_numpy(t),
                'x': torch.from_numpy(x),
                'y': torch.from_numpy(y),
            }
            x = event_data_torch['x']
            y = event_data_torch['y']
            xy_rect = self.rectify_events(x.int(), y.int())
            x_rect = torch.from_numpy(xy_rect[:, 0]).long()
            y_rect = torch.from_numpy(xy_rect[:, 1]).long()
            value = 2*event_data_torch['p']-1
            index = self.width*y_rect + x_rect
            mask = (x_rect < self.width) & (y_rect < self.height)
            event_count[i].put_(index[mask], value[mask], accumulate=True)

        return event_count

    @staticmethod
    def normalize_tensor(event_count):
        mask = torch.nonzero(event_count, as_tuple=True)
        if mask[0].size()[0] > 0:
            mean = event_count[mask].mean()
            std = event_count[mask].std()
            if std > 0:
                event_count[mask] = (event_count[mask] - mean) / std
            else:
                event_count[mask] = event_count[mask] - mean
        return event_count


class SequenceRecurrent(Sequence):
    def __init__(self, seq_path: Path, representation_type: RepresentationType, mode: str = 'test', delta_t_ms: int = 100,
                 num_bins: int = 15, transforms=None, sequence_length=1, name_idx=0, visualize=False, load_gt=False):
        super(SequenceRecurrent, self).__init__(seq_path, representation_type, mode, delta_t_ms, transforms=transforms,
                                                name_idx=name_idx, visualize=visualize, load_gt=load_gt)
        self.crop_size = self.transforms['randomcrop'] if 'randomcrop' in self.transforms else None
        self.sequence_length = sequence_length
        self.valid_indices = self.get_continuous_sequences()

    def get_continuous_sequences(self):
        continuous_seq_idcs = []
        if self.sequence_length > 1:
            for i in range(len(self.timestamps_flow)-self.sequence_length+1):
                diff = self.timestamps_flow[i + self.sequence_length-1] - self.timestamps_flow[i]
                if diff < np.max([100000 * (self.sequence_length-1) + 1000, 101000]):
                    continuous_seq_idcs.append(i)
        else:
            for i in range(len(self.timestamps_flow)-1):
                diff = self.timestamps_flow[i+1] - self.timestamps_flow[i]
                if diff < np.max([100000 * (self.sequence_length-1) + 1000, 101000]):
                    continuous_seq_idcs.append(i)
        return continuous_seq_idcs

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        assert idx >= 0
        assert idx < len(self)

        # Valid index is the actual index we want to load, which guarantees a continuous sequence length
        valid_idx = self.valid_indices[idx]

        sequence = []
        j = valid_idx

        ts_cur = self.timestamps_flow[j]
        # Add first sample
        sample = self.get_data_sample(j)
        sequence.append(sample)

        # Data augmentation according to first sample
        crop_window = None
        flip = None
        if 'crop_window' in sample.keys():
            crop_window = sample['crop_window']
        if 'flipped' in sample.keys():
            flip = sample['flipped']

        for i in range(self.sequence_length-1):
            j += 1
            ts_old = ts_cur
            ts_cur = self.timestamps_flow[j]
            assert(ts_cur-ts_old < 100000 + 1000)
            sample = self.get_data_sample(
                j, crop_window=crop_window, flip=flip)
            sequence.append(sample)

        # Check if the current sample is the first sample of a continuous sequence
        if idx == 0 or self.valid_indices[idx]-self.valid_indices[idx-1] != 1:
            sequence[0]['new_sequence'] = 1
            print("Timestamp {} is the first one of the next seq!".format(
                self.timestamps_flow[self.valid_indices[idx]]))
        else:
            sequence[0]['new_sequence'] = 0

        # random crop
        if self.crop_size is not None:
            i, j, h, w = RandomCrop.get_params(
                sample["event_volume_old"], output_size=self.crop_size)
            keys_to_crop = ["event_volume_old", "event_volume_new",
                            "flow_gt_event_volume_old", "flow_gt_event_volume_new",
                            "flow_gt_next",]

            for sample in sequence:
                for key, value in sample.items():
                    if key in keys_to_crop:
                        if isinstance(value, torch.Tensor):
                            sample[key] = tf.functional.crop(value, i, j, h, w)
                        elif isinstance(value, list) or isinstance(value, tuple):
                            sample[key] = [tf.functional.crop(v, i, j, h, w) for v in value]
        return sequence


class DatasetProvider:
    def __init__(self, dataset_path: Path, representation_type: RepresentationType, delta_t_ms: int = 100, num_bins=4, config=None, visualize=False, transforms=None):
        test_path = Path(os.path.join(dataset_path, 'test'))
        train_path = Path(os.path.join(dataset_path, 'train'))
        assert dataset_path.is_dir(), str(dataset_path)
        assert test_path.is_dir(), str(test_path)
        assert delta_t_ms == 100
        self.config = config
        self.name_mapper_test = []

        if transforms:
            self.transforms = transforms
        else:
            self.transforms = tf.Compose([
                transforms.ToTensor(),  # Convert image to PyTorch tensor
            ])

        # Assemble test sequences
        test_sequences = list()
        for child in test_path.iterdir():
            self.name_mapper_test.append(str(child).split("/")[-1])
            test_sequences.append(Sequence(child, representation_type, 'test', delta_t_ms, num_bins,
                                               name_idx=len(self.name_mapper_test) - 1,
                                               visualize=visualize,
                                               transforms=self.transforms))

        self.test_dataset = torch.utils.data.ConcatDataset(test_sequences)

        # Assemble train sequences
        available_seqs = os.listdir(train_path)

        seqs = available_seqs

        train_sequences: list[Sequence] = []
        for seq in seqs:
            extra_arg = dict()
            train_sequences.append(Sequence(Path(train_path) / seq,
                                   representation_type=representation_type, mode="train",
                                   load_gt=True, **extra_arg, transforms=self.transforms))
            self.train_dataset: torch.utils.data.ConcatDataset[Sequence] = torch.utils.data.ConcatDataset(train_sequences)

    def get_test_dataset(self):
        return self.test_dataset

    def get_train_dataset(self):
        return self.train_dataset

    def get_name_mapping_test(self):
        return self.name_mapper_test

    def summary(self, logger):
        logger.write_line(
            "================================== Dataloader Summary ====================================", True)
        logger.write_line("Loader Type:\t\t" + self.__class__.__name__, True)
        logger.write_line("Number of Voxel Bins: {}".format(
            self.test_dataset.datasets[0].num_bins), True)
        logger.write_line("Number of Train Sequences: {}".format(
            len(self.train_dataset)), True)

def train_collate(sample_list):
    batch = dict()
    for field_name in sample_list[0]:
        if field_name == 'timestamp':
            batch['timestamp'] = [sample[field_name] for sample in sample_list]
        if field_name == 'seq_name':
            batch['seq_name'] = [sample[field_name] for sample in sample_list]
        if field_name == 'new_sequence':
            batch['new_sequence'] = [sample[field_name]
                                     for sample in sample_list]
        if field_name.startswith("event_volume"):
            batch[field_name] = torch.stack(
                [sample[field_name] for sample in sample_list])
        if field_name.startswith("flow_gt") or field_name.startswith('prev_flow_gt') or field_name.startswith('next_flow_gt'):
            if all(field_name in x for x in sample_list):
                batch[field_name] = torch.stack(
                    [sample[field_name][0] for sample in sample_list])
                batch[field_name + '_valid_mask'] = torch.stack(
                    [sample[field_name][1] for sample in sample_list])

    return batch


def rec_train_collate(sample_list):
    seq_length = len(sample_list[0])
    seq_of_batch = []
    for i in range(seq_length):
        seq_of_batch.append(train_collate(
            [sample[i] for sample in sample_list]))
    return seq_of_batch

## `base.py`

In [ ]:
import torch
from torch import nn

class build_resnet_block(nn.Module):
    """
    a resnet block which includes two general_conv2d
    """
    def __init__(self, channels, layers=2, do_batch_norm=False):
        super(build_resnet_block,self).__init__()
        self._channels = channels
        self._layers = layers

        self.res_block = nn.Sequential(*[general_conv2d(in_channels=self._channels,
                                             out_channels=self._channels,
                                             strides=1,
                                             do_batch_norm=do_batch_norm) for i in range(self._layers)])

    def forward(self,input_res):
        inputs = input_res.clone()
        input_res = self.res_block(input_res)
        return input_res + inputs

class upsample_conv2d_and_predict_flow(nn.Module):
    """
    an upsample convolution layer which includes a nearest interpolate and a general_conv2d
    """
    def __init__(self, in_channels, out_channels, ksize=3, do_batch_norm=False):
        super(upsample_conv2d_and_predict_flow, self).__init__()
        self._in_channels = in_channels
        self._out_channels = out_channels
        self._ksize = ksize
        self._do_batch_norm = do_batch_norm

        self.general_conv2d = general_conv2d(
            in_channels=self._in_channels,
            out_channels=self._out_channels,
            ksize=self._ksize,
            strides=1,
            do_batch_norm=self._do_batch_norm,
            padding=0
        )

        self.pad = nn.ReflectionPad2d(
            padding=(
                int((self._ksize - 1) / 2),
                int((self._ksize - 1) / 2),
                int((self._ksize - 1) / 2),
                int((self._ksize - 1) / 2)
            )
        )

        self.predict_flow = general_conv2d(
            in_channels=self._out_channels,
            out_channels=2,
            ksize=1,
            strides=1,
            padding=0,
            activation='tanh'
        )

    def forward(self, conv):
        shape = conv.shape
        conv = nn.functional.interpolate(conv,size=[shape[2]*2,shape[3]*2],mode='nearest')
        conv = self.pad(conv)
        conv = self.general_conv2d(conv)

        flow = self.predict_flow(conv) * 256.

        return torch.cat([conv,flow.clone()], dim=1), flow

def general_conv2d(in_channels,out_channels, ksize=3, strides=2, padding=1, do_batch_norm=False, activation='relu',
                   attention=False):
    """
    a general convolution layer which includes a conv2d, a relu and a batch_normalize
    """
    layers = [nn.Conv2d(in_channels = in_channels,out_channels = out_channels,kernel_size = ksize,
                        stride=strides,padding=padding)]

    if activation == 'relu':
        layers.append(nn.ReLU(inplace=True))

        if do_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels,eps=1e-5,momentum=0.99))

    elif activation == 'tanh':
        layers.append(nn.Tanh())

        if do_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels,eps=1e-5,momentum=0.99))

    if attention:
        # layers.append(Attention(out_channels, heads=2, dim_head=8, dropout=0.1))
        layers.append(BlockSparseAttention(in_dim=out_channels, heads=2, blocksize=16))

    return nn.Sequential(*layers)

## `evflownet.py`

### Attention Mechanisms

For some reason, the self-attention models use more than the 334GB RAM limit that Google Colab's TPU has.

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads=2, dim_head=8, dropout=0.1):
        super().__init__()

        self.dim = dim
        self.dim_head = dim_head
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = math.sqrt(dim_head)

        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

        self.to_q = nn.Linear(dim, inner_dim)
        self.to_k = nn.Linear(dim, inner_dim)
        self.to_v = nn.Linear(dim, inner_dim)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        B, C, H, W = x.size()  # Shape: [B, C, H, W]

        x = x.view(B, C, -1).transpose(1, 2)  # Shape: [B, H*W, C]

        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)

        q = rearrange(q, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)
        k = rearrange(k, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)
        v = rearrange(v, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)

        dots = torch.matmul(q, k.transpose(-2, -1)) / self.scale
        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, "b h n d -> b n (h d)", h=self.heads, d=self.dim_head)

        out = self.to_out(out)

        out = out.transpose(1, 2).view(B, C, H, W)  # Shape: [B, C, H, W]

        return out

In [ ]:
def split_heads(x, heads):
    return torch.transpose(split_states(x, heads), 1, 2)

def merge_heads(x):
    return merge_states(torch.transpose(x, 1, 2))

def split_states(x, heads):
    x_shape = x.size()
    m = x_shape[-1]
    new_x_shape = x_shape[:-1] + (heads, m // heads)
    return torch.reshape(x, new_x_shape)

def merge_states(x):
    x_shape = x.size()
    new_x_shape = x_shape[:-2] + (x_shape[-2] * x_shape[-1],)
    return torch.reshape(x, new_x_shape)

class BlockSparseAttention(nn.Module):
    def __init__(self, in_dim, heads, blocksize=32):
        super(BlockSparseAttention, self).__init__()
        self.heads = heads
        self.blocksize = blocksize
        self.query = nn.Linear(in_dim, in_dim)
        self.key = nn.Linear(in_dim, in_dim)
        self.value = nn.Linear(in_dim, in_dim)
        self.scale = 1.0 / np.sqrt(in_dim // heads)

    def forward(self, x):
        B, C, H, W = x.size()
        x = x.view(B, H * W, C)
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        q = split_heads(q, self.heads)
        k = split_heads(k, self.heads)
        v = split_heads(v, self.heads)

        w = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        mask = torch.zeros_like(w).fill_(-1e9)
        n_ctx = q.size(2)

        for i in range(0, n_ctx, self.blocksize):
            for j in range(0, n_ctx, self.blocksize):
                mask[:, :, i:i + self.blocksize, j:j + self.blocksize] = 0

        w = w + mask
        w = F.softmax(w, dim=-1)

        a = torch.matmul(w, v)
        a = merge_heads(a)

        a = a.view(B, C, H, W)
        return a

### EvFlowNet

In [ ]:
_BASE_CHANNELS = 8 # default: 64

class EVFlowNet(nn.Module):
    def __init__(self, args):
        super(EVFlowNet,self).__init__()
        self._args = args

        self.encoder1 = general_conv2d(
            in_channels=4,
            out_channels=_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm,
            attention=False
        )
        self.encoder2 = general_conv2d(
            in_channels=_BASE_CHANNELS,
            out_channels=2*_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm,
            attention=False
        )
        self.encoder3 = general_conv2d(
            in_channels=2*_BASE_CHANNELS,
            out_channels=4*_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm,
            attention=False
        )
        self.encoder4 = general_conv2d(
            in_channels=4*_BASE_CHANNELS,
            out_channels=8*_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm,
            attention=False
        )

        self.resnet_block = nn.Sequential(*[build_resnet_block(8*_BASE_CHANNELS, do_batch_norm=not self._args.no_batch_norm) for i in range(2)])

        self.decoder1 = upsample_conv2d_and_predict_flow(
            in_channels=16*_BASE_CHANNELS,
            out_channels=4*_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm
        )
        self.decoder2 = upsample_conv2d_and_predict_flow(
            in_channels=8*_BASE_CHANNELS+2,
            out_channels=2*_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm
        )
        self.decoder3 = upsample_conv2d_and_predict_flow(
            in_channels=4*_BASE_CHANNELS+2,
            out_channels=_BASE_CHANNELS,
            do_batch_norm=not self._args.no_batch_norm
        )
        self.decoder4 = upsample_conv2d_and_predict_flow(
            in_channels=2*_BASE_CHANNELS+2,
            out_channels=int(_BASE_CHANNELS/2),
            do_batch_norm=not self._args.no_batch_norm
        )

    def forward(self, inputs: torch.Tensor) -> Dict[str, torch.Tensor]: # [16, 4, 480, 640]

        # encoder
        skip_connections = {}
        inputs = self.encoder1(inputs)
        # inputs = checkpoint(self.encoder1, inputs)
        skip_connections['skip0'] = inputs.clone() # torch.Size([16, 64, 240, 320])

        inputs = self.encoder2(inputs)
        # inputs = checkpoint(self.encoder2, inputs)
        skip_connections['skip1'] = inputs.clone() # torch.Size([16, 128, 120, 160])

        inputs = self.encoder3(inputs)
        # inputs = checkpoint(self.encoder3, inputs)
        skip_connections['skip2'] = inputs.clone() # torch.Size([16, 256, 60, 80])

        inputs = self.encoder4(inputs)
        # inputs = checkpoint(self.encoder4, inputs)
        skip_connections['skip3'] = inputs.clone() # torch.Size([16, 512, 30, 40])

        # transition
        inputs = self.resnet_block(inputs) # torch.Size([16, 512, 30, 40])

        # decoder
        flow_dict = {}
        inputs = torch.cat([inputs, skip_connections['skip3']], dim=1) # torch.Size([16, 1024, 30, 40])
        inputs, flow = self.decoder1(inputs) # inputs: torch.Size([16, 258, 60, 80])
        flow_dict['flow0'] = flow.clone() # torch.Size([16, 2, 60, 80])

        inputs = torch.cat([inputs, skip_connections['skip2']], dim=1) # torch.Size([16, 514, 60, 80])
        inputs, flow = self.decoder2(inputs) # inputs: torch.Size([16, 130, 120, 160])
        flow_dict['flow1'] = flow.clone() # torch.Size([16, 2, 120, 160])

        inputs = torch.cat([inputs, skip_connections['skip1']], dim=1) # torch.Size([16, 258, 120, 160])
        inputs, flow = self.decoder3(inputs) # inputs: torch.Size([16, 66, 240, 320])
        flow_dict['flow2'] = flow.clone() # torch.Size([16, 2, 240, 320])

        inputs = torch.cat([inputs, skip_connections['skip0']], dim=1) # torch.Size([16, 130, 240, 320])
        inputs, flow = self.decoder4(inputs) # inputs: torch.Size([16, 34, 480, 640])
        flow_dict['flow3'] = flow.clone() # torch.Size([16, 2, 480, 640])

        return flow_dict

## Image Preprocessing

In [ ]:
class HistogramEqualization:
    def __call__(self, image):
        print(np.array(image).shape)

        # Needs to have 3 channels or color passed in
        # runs into the problem of loss of information in dimensionality reduction
        np_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

#         np_image = np.array(image)
        eq_image = cv2.equalizeHist(np_image)
        return Image.fromarray(eq_image)

# combined_transform = CombinedTransform(transform=tf.Compose([
#     tf.GaussianBlur(kernel_size=(5, 5)),
# ]))

In [ ]:
import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image, to_tensor
import torch.nn.functional as nn_F

class DSECPreprocess:
    def __init__(self, kernel_size=5, sigma=1.0):
        self.kernel_size = kernel_size
        self.sigma = sigma

    def __call__(self, img):
        return self.preprocess_image(img)

    def preprocess_image(self, img):
        img = img.to(torch.float32)
        img = img.mean(dim=0, keepdim=True)
        img = (img - img.min()) / (img.max() - img.min() + 1e-5)

        img = self.gaussian_blur(img)
        # img = self.histogram_equalization(img)

        return img

    def gaussian_blur(self, img):
        _, channels, height, width = img.shape
        # img = img.unsqueeze(0)  # Add batch dimension

        kernel = self.get_gaussian_kernel(self.kernel_size, self.sigma)
        kernel = kernel.expand(channels, 1, -1, -1)  # Expand to match the number of channels

        img = img.double() # expects double (torch.float64)
        kernel = kernel.double()
        img = nn_F.conv2d(img, kernel, padding=self.kernel_size // 2, groups=channels)

        # img = img.squeeze(0)  # Remove batch dimension
        return img

    def get_gaussian_kernel(self, kernel_size, sigma):
        k = torch.arange(kernel_size).float()
        x = k - (kernel_size - 1) / 2
        gauss = torch.exp(-0.5 * (x / sigma)**2)
        gauss = gauss / gauss.sum()
        kernel = gauss[:, None] * gauss[None, :]
        return kernel[None, None, :, :]

    def histogram_equalization(self, img):
        batch_size, channels, height, width = img.shape

        img1 = img[:, :2]  # First two channels
        img2 = img[:, 2:]  # Last two channels

        equalized_images1 = []
        equalized_images2 = []

        for i in range(batch_size):
            img_i1 = to_pil_image(img1[i])
            img_i1 = transforms.functional.equalize(img_i1)
            img_i1 = to_tensor(img_i1)
            equalized_images1.append(img_i1)

            img_i2 = to_pil_image(img2[i])
            img_i2 = transforms.functional.equalize(img_i2)
            img_i2 = to_tensor(img_i2)
            equalized_images2.append(img_i2)

        equalized_img1 = torch.stack(equalized_images1, dim=0)
        equalized_img2 = torch.stack(equalized_images2, dim=0)
        return equalized_img1, equalized_img2

In [ ]:
a = torch.rand((16, 4, 480, 640))
b = DSECPreprocess()(a)
b.shape

torch.Size([1, 4, 480, 640])

In [ ]:
class CombinedTransform:
    def __init__(self, transform=tf.Compose([ tf.ToTensor() ])):
        self.transform = transform

    def __call__(self, flow_dict):
        seed = np.random.randint(2147483647)

        flow_columns = ['event_volume', 'flow_gt', 'prev_flow_gt', 'next_flow_gt']
        flow_columns = [c for c in flow_columns if c in list(flow_dict.keys())]

        for col in flow_columns:
            torch.manual_seed(seed)

            if type(flow_dict[col]) == list:
                flow_dict[col] = [ self.transform(img) for img in flow_dict[col] ]
            else:
                flow_dict[col] = self.transform(flow_dict[col])

        return flow_dict

In [ ]:
combined_transform = CombinedTransform(transform=DSECPreprocess(kernel_size=5, sigma=1.0))

## `main.py`

Instead of changing the `Sequence`, just change how the data are loaded. Or iterate over the dataloader so that you use many images at once.

In [ ]:
class RepresentationType(Enum):
    VOXEL = auto()
    STEPAN = auto()

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

def compute_epe_error(pred_flow: torch.Tensor, gt_flow: torch.Tensor):
    '''
    end-point-error (ground truthと予測値の二乗誤差)を計算
    pred_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 予測したオプティカルフローデータ
    gt_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 正解のオプティカルフローデータ
    '''
    epe = torch.mean(torch.mean(torch.norm(pred_flow - gt_flow, p=2, dim=1), dim=(1, 2)), dim=0)
    return epe

def save_optical_flow_to_npy(flow: torch.Tensor, file_name: str):
    '''
    optical flowをnpyファイルに保存
    flow: torch.Tensor, Shape: torch.Size([2, 480, 640]) => オプティカルフローデータ
    file_name: str => ファイル名
    '''
    np.save(f"{file_name}.npy", flow.cpu().numpy())

In [ ]:
args = cfg
set_seed(args.seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''
    ディレクトリ構造:

    data
    ├─test
    |  ├─test_city
    |  |    ├─events_left
    |  |    |   ├─events.h5
    |  |    |   └─rectify_map.h5
    |  |    └─forward_timestamps.txt
    └─train
        ├─zurich_city_11_a
        |    ├─events_left
        |    |       ├─ events.h5
        |    |       └─ rectify_map.h5
        |    ├─ flow_forward
        |    |       ├─ 000134.png
        |    |       |.....
        |    └─ forward_timestamps.txt
        ├─zurich_city_11_b
        └─zurich_city_11_c
    '''

# ------------------
#    Dataloader
# ------------------

loader = DatasetProvider(
    dataset_path=Path(args.dataset_path),
    representation_type=RepresentationType.VOXEL,
    delta_t_ms=100,
    num_bins=4,
    transforms=combined_transform # Custom class
)
train_set = loader.get_train_dataset()
test_set = loader.get_test_dataset()

# def split_train_valid(dataset):
#     train_indices = []
#     valid_indices = []
#     for idx in range(len(dataset)):
#         sample = dataset[idx]
#         if 'flow_gt_valid_mask' in sample and sample['flow_gt_valid_mask'].all():
#             valid_indices.append(idx)
#         else:
#             train_indices.append(idx)
#     train_subset = torch.utils.data.Subset(dataset, train_indices)
#     valid_subset = torch.utils.data.Subset(dataset, valid_indices)
#     return train_subset, valid_subset

# train_set_split, valid_set_split = split_train_valid(train_set)

collate_fn = train_collate
train_data = DataLoader(train_set, # train_set_split
                        batch_size=args.data_loader.train.batch_size,
                        shuffle=args.data_loader.train.shuffle,
                        collate_fn=collate_fn,
                        drop_last=False,
                        num_workers=os.cpu_count(),
                        pin_memory=True)
# valid_data = DataLoader(valid_set_split,
#                         batch_size=args.data_loader.train.batch_size,
#                         shuffle=args.data_loader.train.shuffle,
#                         collate_fn=collate_fn,
#                         drop_last=False,
#                         num_workers=os.cpu_count(),
#                         pin_memory=True)
test_data = DataLoader(test_set,
                       batch_size=args.data_loader.test.batch_size,
                       shuffle=args.data_loader.test.shuffle,
                       collate_fn=collate_fn,
                       drop_last=False,
                       num_workers=os.cpu_count(),
                       pin_memory=True)

'''
train data:
    Type of batch: Dict
    Key: seq_name, Type: list
    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
    Key: flow_gt, Type: torch.Tensor, Shape: torch.Size([Batch, 2, 480, 640]) => オプティカルフローデータのバッチ
    Key: flow_gt_valid_mask, Type: torch.Tensor, Shape: torch.Size([Batch, 1, 480, 640]) => オプティカルフローデータのvalid. ベースラインでは使わない

test data:
    Type of batch: Dict
    Key: seq_name, Type: list
    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
'''

'\ntrain data:\n    Type of batch: Dict\n    Key: seq_name, Type: list\n    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ\n    Key: flow_gt, Type: torch.Tensor, Shape: torch.Size([Batch, 2, 480, 640]) => オプティカルフローデータのバッチ\n    Key: flow_gt_valid_mask, Type: torch.Tensor, Shape: torch.Size([Batch, 1, 480, 640]) => オプティカルフローデータのvalid. ベースラインでは使わない\n\ntest data:\n    Type of batch: Dict\n    Key: seq_name, Type: list\n    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ\n'

In [ ]:
# ------------------
#       Model
# ------------------
model = EVFlowNet(args.train).to(device)

# ------------------
#   optimizer
# ------------------
optimizer = torch.optim.Adam(model.parameters(), lr=args.train.initial_learning_rate, weight_decay=args.train.weight_decay)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=args.train.learning_rate_decay)

loss_fn = TotalLoss(smoothness_weight=0.5)

In [ ]:
# num_epochs = args.train.epochs
num_epochs = 1

epe_losses = [[] for _ in range(num_epochs)]
overall_losses = [[] for _ in range(num_epochs)]

In [ ]:
BATCH_CONCAT = 2

In [ ]:
# ------------------
#   Start training
# ------------------
model.train()

for epoch in range(num_epochs):

    total_loss = 0
    prev_event_volumes = [] # Acts as a queue

    print("on epoch: {}".format(epoch + 1))
    for i, batch in enumerate(tqdm(train_data)):

        try:
            batch: Dict[str, Any]

            event_image = batch["event_volume"].to(device) # [B, 4, 480, 640]
            ground_truth_flow = batch["flow_gt"].to(device) # [B, 2, 480, 640]

            prev_event_volumes.append(event_image)

            prev_ground_truth_flow = batch['prev_flow_gt'].to(device) # [B, 2, 480, 640]
            next_ground_truth_flow = batch['next_flow_gt'].to(device) # [B, 2, 480, 640]

            # Temporal averaging
            # avg_flow_gt = torch.mean(torch.cat([
            #     ground_truth_flow.unsqueeze(1),
            #     prev_ground_truth_flow.unsqueeze(1),
            #     next_ground_truth_flow.unsqueeze(1)
            # ], dim=1), dim=1)

            try:
                avg_event_image = torch.mean(torch.stack(prev_event_volumes, dim=0), dim=0)
            except RuntimeError:
                current_batch_size, C, H, W = event_image.shape
                zero_padding = torch.zeros((16 - current_batch_size, C, H, W), dtype=event_image.dtype, device=event_image.device)
                padded_event_image = torch.cat([event_image, zero_padding], dim=0)

                prev_event_volumes = prev_event_volumes[:-1] + [padded_event_image]
                avg_event_image = torch.mean(torch.stack(prev_event_volumes, dim=0), dim=0)

            print(avg_event_image.shape)
            flow_dict = model(avg_event_image) # 'flow3' is of shape [B, 2, 480, 640]

            epe_loss: torch.Tensor = compute_epe_error(flow_dict['flow3'], ground_truth_flow)
            overall_loss: torch.Tensor = loss_fn(flow_dict,
                                         prev_ground_truth_flow,
                                         next_ground_truth_flow,
                                         model)

            print(f"batch {i} OVERALL LOSS: {overall_loss.item()}")
            print(f"batch {i} EPE LOSS: {epe_loss.item()}")
            overall_losses[epoch].append(overall_loss.item())
            epe_losses[epoch].append(epe_loss.item())

            optimizer.zero_grad()
            epe_loss.backward() # Change this to which loss function is to be updated
            optimizer.step()

            total_loss += epe_loss.item() # This too

            if len(prev_event_volumes) >= BATCH_CONCAT:
                prev_event_volumes.pop()

        except KeyboardInterrupt:
            current_time = time.strftime("%Y%m%d%H%M%S")
            model_path = f"models/model_{current_time}.pth" # /kaggle/input/
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")

            raise SystemExit("KeyboardInterrupt")

    scheduler.step()

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_data)}')

on epoch: 1


  0%|          | 0/126 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b3c4151b9a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b3c4151b9a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    Exception ignored in: self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x7b3c4151b9a0>

Traceback (most recent

torch.Size([16, 1, 480, 640])


  0%|          | 0/126 [00:35<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 4, 3, 3], expected input[16, 1, 480, 640] to have 4 channels, but got 1 channels instead

In [ ]:
import matplotlib.pyplot as plt

# epe_losses = list(map(lambda x: x.item(), epe_losses[0]))
# overall_losses = list(map(lambda x: x.item(), overall_losses[0]))

plt.figure(figsize=(16, 9))
len_x = min(len(epe_losses), len(overall_losses))

plt.plot(epe_losses[:len_x])
plt.plot(overall_losses[:len_x])

plt.xlabel('Batch Number')
plt.ylabel('Loss')

plt.grid()
plt.legend()

plt.show()

In [ ]:
import time
# Create the directory if it doesn't exist
# if not os.path.exists('checkpoints'):
#     os.makedirs('checkpoints')

current_time = time.strftime("%Y%m%d-%H%M%S")
model_path = f"models/model_{current_time}_epoch4.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
model.eval()
flow: torch.Tensor = torch.tensor([]).to(device)

prev_event_volumes = [] # Acts as a queue

with torch.no_grad():
    print("start test")
    for batch in tqdm(test_data):
        batch: Dict[str, Any]

        event_image = batch["event_volume"].to(device)
        prev_event_volumes.append(event_image)

        avg_event_image = torch.mean(torch.stack(prev_event_volumes, dim=0), dim=0)

        batch_flow = model(avg_event_image) # [1, 2, 480, 640]
        flow = torch.cat((flow, batch_flow['flow3']), dim=0)  # [N, 2, 480, 640]

        if len(prev_event_volumes) >= BATCH_CONCAT:
            prev_event_volumes.pop()

    print("test done")
# ------------------
#  save submission
# ------------------
current_time = time.strftime("%Y%m%d-%H%M%S")
save_optical_flow_to_npy(flow, f'submissions/submission_{current_time}')